# Data Cleaning & Visualization Pipeline
This notebook demonstrates the process:
**Raw CSV → Cleaned Dataset → Visual Insights**


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/raw_orders.csv")
df.head()